In [21]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import pandas as pd

In [22]:
def add_experiment_parameters_to_file(file_path):
    # Read the CSV file into a pandas dataframe
    df = pd.read_csv(file_path)

    # Extract terms from the file name
    file_name = file_path.split("/")[-1]  # Extracting just the file name
    terms = file_name.replace(".csv", "").split("_")
    print(terms)

    # Append each term to a new column in the dataframe
    df["ticker"] = terms[0]
    df["rsi14_entry"] = terms[1]
    df["rsi14_exit_high"] = terms[2]
    df["rsi14_exit_low"] = terms[3]
    df["max_trades"] = terms[4]
    # put these columns at the front of the dataframe
    cols = df.columns.tolist()
    cols = cols[-5:] + cols[:-5]
    df = df[cols]


    return df

# Example usage
file_path = "../../temp/rsi_backtests/AAPL_40_60_20_1.csv"
trades = add_experiment_parameters_to_file(file_path)

# Display the resulting dataframe
trades.head(20)


['AAPL', '40', '60', '20', '1']


,ticker,rsi14_entry,rsi14_exit_high,rsi14_exit_low,max_trades,Date,Direction,Entry,Exit,TimeEnter,TimeExit,Gain%,Gain$,ExitCondition
0,AAPL,40,60,20,1,2022-09-30,Long,141.7300,141.9900,10:13:00,10:28:00,0.18,0.26,MaxProfit
1,AAPL,40,60,20,1,2022-10-03,Long,138.6300,138.5100,09:32:00,09:37:00,-0.09,-0.12,MaxProfit
2,AAPL,40,60,20,1,2022-10-04,Long,144.9900,144.9999,09:34:00,09:47:00,0.01,0.01,MaxProfit
3,AAPL,40,60,20,1,2022-10-05,Long,143.3476,143.6550,10:36:00,10:45:00,0.21,0.31,MaxProfit
4,AAPL,40,60,20,1,2022-10-06,Long,146.0795,146.0000,10:14:00,10:23:00,-0.05,-0.08,MaxProfit
5,AAPL,40,60,20,1,2022-10-07,Long,142.5250,142.7250,09:32:00,09:42:00,0.14,0.20,MaxProfit
6,AAPL,40,60,20,1,2022-10-10,Long,139.1901,138.8400,09:48:00,10:09:00,-0.25,-0.35,MaxProfit
7,AAPL,40,60,20,1,2022-10-11,Long,139.7100,138.8700,09:30:00,10:22:00,-0.60,-0.84,MaxProfit
8,AAPL,40,60,20,1,2022-10-12,Long,139.0000,139.6250,09:33:00,09:40:00,0.45,0.62,MaxProfit
9,AAPL,40,60,20,1,2022-10-13,Long,136.2000,136.1400,10:20:00,10:42:00,-0.04,-0.06,MaxProfit


In [23]:
import plotly.express as px

# line chart of running gain%
def chart_gain_percent(trades):
    trades["RunningGain%"] = trades["Gain%"].cumsum()
    fig = px.line(trades, y="RunningGain%", title="Running Gain%")
    fig.show()

In [24]:
def rsi_ranged_backtest_single_pnl(df):
    res = pd.DataFrame()
    res.loc[0, 'Ticker'] = df['ticker'].iloc[0]
    res.loc[0, 'StartDate'] = df['Date'].min()
    res.loc[0, 'EndDate'] = df['Date'].max()
    res.loc[0, 'RSI Entry'] = df["rsi14_entry"].iloc[0]
    res.loc[0, 'RSI Exit High'] = df["rsi14_exit_high"].iloc[0]
    res.loc[0, 'RSI Exit Stoploss'] = df["rsi14_exit_low"].iloc[0]
    res.loc[0, 'MaxTrades'] = df["max_trades"].iloc[0]
    res.loc[0, 'TotalTrades'] = len(df)
    res.loc[0, 'TotalPercentGain'] = round(df['Gain%'].sum(), 2)
    res.loc[0, 'AveragePercentGain'] = round(df['Gain%'].mean(), 2)
    res.loc[0, 'MaxPercentGain'] = round(df['Gain%'].max(), 2)
    res.loc[0, 'MinPercentGain'] = round(df['Gain%'].min(), 2)

    return res

In [25]:
rsi_ranged_backtest_single_pnl(trades)

,Ticker,StartDate,EndDate,RSI Entry,RSI Exit High,RSI Exit Stoploss,MaxTrades,TotalTrades,TotalPercentGain,AveragePercentGain,MaxPercentGain,MinPercentGain
0,AAPL,2022-09-30,2023-09-29,40,60,20,1,251.0,-0.92,-0.0,1.5,-2.8


In [26]:
all_results = pd.DataFrame(
    columns=['Ticker', 'StartDate', 'EndDate', 'RSI Entry', 'RSI Exit High', 'RSI Exit Stoploss', 'MaxTrades',
             'TotalTrades', 'TotalPercentGain', 'AveragePercentGain', 'MaxPercentGain', 'MinPercentGain'])

for e in os.listdir("../../temp/rsi_backtests/"):
    if e.endswith(".csv"):
        file_path = "../../temp/rsi_backtests/" + e
        trades = add_experiment_parameters_to_file(file_path)
        pnl = rsi_ranged_backtest_single_pnl(trades)
        all_results = pd.concat([all_results, pnl], axis=0)

['AAPL', '50', '60', '40', '2']
['AAPL', '60', '80', '40', '1']
['QQQ', '40', '60', '20', '2']
['META', '50', '60', '40', '4']
['SPY', '50', '70', '30', '2']
['AMZN', '40', '60', '20', '1']
['SPY', '50', '70', '30', '3']
['AAPL', '50', '60', '40', '3']
['TSLA', '60', '80', '40', '4']
['AAPL', '60', '80', '40', '2']
['AAPL', '50', '60', '40', '1']
['QQQ', '40', '60', '20', '1']
['META', '60', '80', '40', '4']
['AMZN', '40', '60', '20', '2']
['SPY', '50', '70', '30', '1']
['AMZN', '40', '60', '20', '3']
['AAPL', '60', '80', '40', '3']
['TSLA', '50', '60', '40', '4']
['AAPL', '50', '60', '40', '4']
['TSLA', '60', '80', '40', '3']
['META', '60', '80', '40', '1']
['META', '50', '60', '40', '2']
['MSFT', '40', '60', '20', '1']
['SPY', '50', '70', '30', '4']
['META', '50', '60', '40', '3']
['TSLA', '50', '60', '40', '1']
['TSLA', '60', '80', '40', '2']
['AAPL', '60', '80', '40', '4']
['TSLA', '50', '60', '40', '3']
['META', '50', '60', '40', '1']
['META', '60', '80', '40', '2']
['MSFT', '40',

In [30]:
# sort by total gain
all_results.sort_values(by=['TotalPercentGain'], ascending=False, inplace=True)

all_results.head(20)

,Ticker,StartDate,EndDate,RSI Entry,RSI Exit High,RSI Exit Stoploss,MaxTrades,TotalTrades,TotalPercentGain,AveragePercentGain,MaxPercentGain,MinPercentGain
0,META,2022-09-30,2023-09-29,40,60,20,2,502.0,23.84,0.05,2.76,-2.31
0,META,2022-09-30,2023-09-29,50,70,30,4,1004.0,22.81,0.02,3.55,-2.10
0,META,2022-09-30,2023-09-29,50,70,30,2,502.0,22.45,0.04,3.55,-2.10
0,VXX,2022-09-30,2023-09-29,50,60,40,2,502.0,19.21,0.04,6.30,-3.50
0,META,2022-09-30,2023-09-29,40,60,20,3,753.0,17.65,0.02,2.76,-2.31
0,VXX,2022-09-30,2023-09-29,50,70,30,1,251.0,17.15,0.07,2.83,-2.21
0,META,2022-09-30,2023-09-29,50,70,30,3,753.0,16.70,0.02,3.55,-2.10
0,VXX,2022-09-30,2023-09-29,50,70,30,3,753.0,16.06,0.02,3.99,-3.30
0,META,2022-09-30,2023-09-29,50,70,30,1,251.0,16.00,0.06,2.08,-2.10
0,META,2022-09-30,2023-09-29,40,60,20,1,251.0,14.67,0.06,2.76,-2.00


In [29]:
# histogram of total percent gain
fig = px.histogram(all_results, x="TotalPercentGain", title="Total Percent Gain", nbins=50)
fig.show()

In [ ]:
all_results["TotalPercentGain"].describe()